# 02. 제스처 모델 학습
### 3D CNN (r3d_18) 기반 강조 구간 검출
- CSV에서 라벨 로드
- Negative 구간 자동 생성
- 학습 및 검증

In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from config import GESTURE_CONFIG, GESTURE_MODEL_PATH, ensure_dirs
from utils import (
    prepare_all_entries, train_val_split,
    VideoEmphasisDataset,
    create_gesture_model,
    train_one_epoch, evaluate
)

ensure_dirs()

In [3]:
LABEL_CSV = "/home/stu/ai_project/영상csv/최종csv.csv"  
clip_len = GESTURE_CONFIG["clip_len"]
resize_hw = GESTURE_CONFIG["resize_hw"]
batch_size = GESTURE_CONFIG["batch_size"]
num_epochs = GESTURE_CONFIG["num_epochs"]
lr = GESTURE_CONFIG["lr"]

print(f"clip_len: {clip_len}, batch_size: {batch_size}, epochs: {num_epochs}")

clip_len: 16, batch_size: 180, epochs: 10


## 1. 데이터 준비

In [4]:
# CSV 로드 + Negative 생성
entries_all = prepare_all_entries(LABEL_CSV, num_neg_per_pos=3, shuffle=True)

TypeError: prepare_all_entries() got an unexpected keyword argument 'num_neg_per_pos'

In [ ]:
# Train / Val 분할
train_entries, val_entries = train_val_split(entries_all, split_ratio=0.8)

In [ ]:
# Dataset 생성
train_dataset = VideoEmphasisDataset(train_entries, clip_len=clip_len, resize_hw=resize_hw)
val_dataset = VideoEmphasisDataset(val_entries, clip_len=clip_len, resize_hw=resize_hw)

print(f"Train dataset: {len(train_dataset)}")
print(f"Val dataset: {len(val_dataset)}")

Train dataset: 7653
Val dataset: 1914


In [ ]:
# 샘플 확인
x0, y0 = train_dataset[0]
print(f"x shape: {x0.shape}")  
print(f"y: {y0}")

x shape: torch.Size([3, 32, 112, 112])
y: 0


In [ ]:
# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

# 배치 확인
batch_x, batch_y = next(iter(train_loader))
print(f"batch_x: {batch_x.shape}")  
print(f"batch_y: {batch_y}")

batch_x: torch.Size([180, 3, 32, 112, 112])
batch_y: tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1])


## 2. 모델 생성

In [ ]:
# 모델 생성 (r3d_18 with Kinetics400 pretrained)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_gesture_model(num_classes=2, pretrained=True)
if torch.cuda.device_count() > 1:
        print(f"🔥 {torch.cuda.device_count()}개의 GPU를 발견했습니다! DataParallel을 적용합니다.")
        model = nn.DataParallel(model)
model = model.to(device)

pos_count = sum(1 for e in train_entries if e['label'] == 1)
neg_count = sum(1 for e in train_entries if e['label'] == 0)

print(f"Train set - Positive: {pos_count}, Negative: {neg_count}")

if neg_count > 0 and pos_count > 0:
    pos_weight = neg_count / pos_count
    class_weights = torch.tensor([1.0, pos_weight]).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    print(f"Using class weights: [neg=1.0, pos={pos_weight:.2f}]")
else:
    criterion = nn.CrossEntropyLoss()
    print("Using standard CrossEntropyLoss (no class weights)")

🔥 2개의 GPU를 발견했습니다! DataParallel을 적용합니다.
Train set - Positive: 2324, Negative: 5329
Using class weights: [neg=1.0, pos=2.29]


In [ ]:
# 손실함수 / 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
print(f"Optimizer: AdamW (lr={lr}, weight_decay=1e-4)")

Optimizer: AdamW (lr=0.0001, weight_decay=1e-4)


## 3. 학습

In [ ]:
best_val_acc = 0.0
save_path = GESTURE_MODEL_PATH

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    print(f"[Epoch {epoch+1}/{num_epochs}] "
          f"train_loss={train_loss:.4f}, train_acc={train_acc:.3f} | "
          f"val_loss={val_loss:.4f}, val_acc={val_acc:.3f}")

    # Best 모델 저장
    if val_acc >= best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        torch.save(model.state_dict(), save_path)
        print(f"  -> best model saved to: {save_path}")

print(f"\n최종 best val acc: {best_val_acc:.4f}")

[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref short failure
[h264 @ 0x2d27dbc0] mmco: unref sh

[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref short failure
[h264 @ 0x2be8bbc0] mmco: unref sh

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 96, in _worker
    output = module(*input, **kwargs)
  File "/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/stu/ai_project/utils/models.py", line 52, in forward
    features = self.backbone(x)
  File "/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/stu/anaconda3/envs/ai_project_env/lib/python3.10/site-packages/t

## 4. 학습 완료!
모델이 저장되었습니다. 이제 `06_inference.ipynb`에서 추론할 수 있습니다.